In [ ]:
# Basic imports

import sys

import numpy as np

import xarray as xr

import matplotlib.pyplot as plt

from IPython.display import HTML

import datetime
from cftime import num2date

import cmocean  # for nice oceanographic colourmaps

#JUPYTER notebook magics
%matplotlib inline 

In [ ]:
%%bash

# We tune the model and forcing using these modules
/home/nextsimdg/build/nextsim --help-config WintonAlbedo
/home/nextsimdg/build/nextsim --help-config FluxConfiguredOcean
/home/nextsimdg/build/nextsim --help-config ThermoWinton

In [ ]:
# Create initial conditions using the make_init_base.py class found in nextsimdg/run

sys.path.append('../nextsimdg/run')

from make_init_base import initMaker

# Single column. Resolution doesn't matter, because we're not using the dynamics
nfirst = 1
nsecond = 1
nLayers = 3
resolution = 50

# Initialise the object
initializer = initMaker("init_column.nc", nfirst, nsecond, nLayers, resolution)

ice_salinity = 5.  # should match Ice::s in constants.hpp
mu = -0.055  # should match Water::mu in constants.hpp
ocean_temperature = -1.54
ocean_salinity = ocean_temperature / mu

# Set all the basic variables
initializer.mask[:, :] = 1
initializer.cice[:, :] = 1.
initializer.hice[:, :] = 2.
initializer.hsnow[:, :] = 0.3
initializer.sss[:, :] = ocean_salinity
initializer.sst[:, :] = ocean_temperature
initializer.tice[:, :, :] = ice_salinity * mu

"""
In a normal script, the file is written when initializer goes out of scope - but in 
Jupyter, we need to call __writeFile__ explicitly
"""
initializer.__writeFile__()

In [ ]:
%%bash

# Run the model with the config file for a single column setup
time /home/nextsimdg/build/nextsim --config-file config_files/config_column.cfg

In [ ]:
# Load the NetCDF file
ds = xr.open_dataset("thermo.diagnostic.nc", group="/data")
print(ds)

In [ ]:
# Create sensible dates to use
time = ds['time']
time_vals = num2date(time.values, units='seconds since 1970-01-01', calendar='gregorian')
time_index = 23

In [ ]:
# Calculate ice draught for a nicer visualisation
rho = 917
rhoSnow = 330
rhoOcean = 1025

# Throw away unneeded data and dimensions
hice = np.squeeze(ds['hice'].isel(dg_comp=0))
hsnow = np.squeeze(ds['hsnow'])
tice = np.squeeze(ds['tice'])

iceDraught = (hice * rho + hsnow * rhoSnow) / rhoOcean

In [ ]:
# Some simple diagnostics
print('hice  max: {0:0.2f}, min: {1:0.2f}, mean: {2:0.2f}'.format(hice.max(), hice.min(), hice.mean()))
print('hsnow max: {0:0.2f}, min: {1:0.2f}, mean: {2:0.2f}'.format(hsnow.max(), hsnow.min(), hsnow.mean()))

In [ ]:
# Figure showing temperature evolution, cf,. Winton (2000) figure 2
plt.figure()
plt.plot([0, len(ds['hice'])], [0, 0], 'k--')
plt.plot(tice[:, 0], 'k', label="Surface")
plt.plot(tice[:, 1], label="T1")
plt.plot(tice[:, 2], label="T2")
plt.xlabel("Day of year")
plt.ylabel("Temperature [°C]")
plt.legend()
plt.show()

In [ ]:
# Figure showing thickness evolution, cf. Winton (2000) figure 2
plt.figure()
plt.plot([0, len(hice)], [0, 0], 'k--')
plt.plot(hice - iceDraught, 'b', label="Ice")
plt.plot(hice + hsnow - iceDraught, 'k', label="Snow")
plt.plot(-iceDraught, 'b')
plt.xlabel("Day of year")
plt.ylabel("Height over sea level [m]")
plt.legend()
plt.show()

In [ ]:
ds.close()